Database 1: Samples
===================

After fitting a large suite of strong lens data, we can use the aggregator to load the database's results. We can then
manipulate, interpret and visualize them using a Python script or Jupyter notebook.

This script uses the results generated by the script `/autolens_workspace/database/tutorial_0_model_fits.py`, which
fitted 3 simulated strong lenses with:

 - An `EllIsothermal` `MassProfile` for the lens galaxy's mass.
 - An `EllSersic` `LightProfile` representing a bulge for the source galaxy's light.

__Samples__

This script covers how to manipulate the `Samples` object returned from a *PyAutoLens* model-fit, which you have most
likely already encountered when analysing the results of a model-fit. Nevertheless, we'll also learn how to use the
`Aggregator`!

In [1]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

import os
from os import path
import autofit as af

/mnt/c/Users/Jammy/Code/PyAuto/autolens_workspace
Working Directory has been set to `/mnt/c/Users/Jammy/Code/PyAuto/autolens_workspace`


We now load the results in the `output` folder into a sqlite database using the `Aggregator`. We simply point to the 
path where we want the database to be created and add the directory `autolens_workspace/output/database`.

Checkout the output folder, you should see a `database.sqlite` file which contains the model-fits to the 3 `Gaussian`
datasets.

In [2]:
# from autofit.database.aggregator import Aggregator
#
# database_file = path.join("output", "database", "database.sqlite")
#
# if path.isfile(database_file):
#     os.remove(database_file)
#
# agg = Aggregator.from_database(path.join(database_file))
# agg.add_directory(path.join("output", "database"))

agg = af.Aggregator(directory=path.join("output", "database"))

Aggregator loading search_outputs... could take some time.

 A total of 3 search_outputs and results were found.


Before using the aggregator to inspect results, let me quickly cover Python generators. A generator is an object that 
iterates over a function when it is called. The aggregator creates all of the objects that it loads from the database 
as generators (as opposed to a list, or dictionary, or other Python type).

Why? Because lists and dictionaries store every entry in memory simultaneously. If you fit many datasets, this will use 
a lot of memory and crash your laptop! On the other hand, a generator only stores the object in memory when it is used; 
Python is then free to overwrite it afterwards. Thus, your laptop won't crash!

There are two things to bare in mind with generators:

 1) A generator has no length and to determine how many entries it contains you first must turn it into a list.

 2) Once we use a generator, we cannot use it again and need to remake it. For this reason, we typically avoid 
 storing the generator as a variable and instead use the aggregator to create them on use.

We can now create a `samples` generator of every fit. As we saw in the `result.py` example scripts, an instance of 
the `Samples` class acts as an interface to the results of the non-linear search.

In [3]:
samples_gen = agg.values("samples")

When we print this the length of this generator converted to a list of outputs we see 3 different NestSamples 
instances. These correspond to each fit of each search to each of our 3 images.

In [4]:
print("NestedSampler Samples: \n")
print(samples_gen)
print()
print("Total Samples Objects = ", len(list(samples_gen)), "\n")

NestedSampler Samples: 


Total Samples Objects =  3 



The `Samples` class contains all the parameter samples, which is a list of lists where:
 
 - The outer list is the size of the total number of samples.
 - The inner list is the size of the number of free parameters in the fit.

In [5]:
for samples in agg.values("samples"):

    print("All parameters of the very first sample")
    print(samples.parameters[0])
    print("The third parameter of the tenth sample")
    print(samples.parameters[9][2])

print("Samples: \n")
print(agg.values("samples"))
print()
print("Total Samples Objects = ", len(list(agg.values("samples"))), "\n")

All parameters of the very first sample
[0.0006647790268157113, -0.0008107860682720678, 0.052109613257215234, -0.0006467150908085629, 1.5906877339407972, -0.0013374371833814297, 0.0008828793979972521, 0.05248372334703605, 1.7935235746917366e-05, 0.9990944433441538, 0.8000043696391769, 4.000215797284688]
The third parameter of the tenth sample
0.0523757155849939
All parameters of the very first sample
[0.0008418500087071668, -0.0018494365630365975, 0.05297879659500896, -0.0025922199300690554, 1.6070091230242256, 0.0006299194045255432, -0.001345677866615372, 0.0524705170877404, -0.00023225116351456826, 0.9999154888682447, 0.800179350275853, 3.999420469711665]
The third parameter of the tenth sample
0.05252941004467598
All parameters of the very first sample
[5.6249503742175085e-05, -0.0006579031074579729, 0.0525197465485813, -4.828412109195851e-05, 1.5972112492393538, -0.0014170644776855539, 0.0005011698693998819, 0.052556442020862196, -5.375302382652836e-05, 0.9997608096473558, 0.799317

The `Samples` class contains the log likelihood, log prior, log posterior and weights of every sample, where:

 - The log likelihood is the value evaluated from the likelihood function (e.g. -0.5 * chi_squared + the noise  
 normalization).
    
 - The log prior encodes information on how the priors on the parameters maps the log likelihood value to the log
 posterior value.
      
 - The log posterior is log_likelihood + log_prior.
    
 - The weight gives information on how samples should be combined to estimate the posterior. The weight values 
 depend on the sampler used, for example for MCMC they will all be 1`s.

In [6]:
for samples in agg.values("samples"):
    print("log(likelihood), log(prior), log(posterior) and weight of the tenth sample.")
    print(samples.log_likelihoods[9])
    print(samples.log_priors[9])
    print(samples.log_posteriors[9])
    print(samples.weights[9])

log(likelihood), log(prior), log(posterior) and weight of the tenth sample.
-1e+99
4.916758603159641
-1e+99
0.0
log(likelihood), log(prior), log(posterior) and weight of the tenth sample.
-1e+99
8.57865100984459
-1e+99
0.0
log(likelihood), log(prior), log(posterior) and weight of the tenth sample.
-1e+99
5.278149702855675
-1e+99
0.0


We can use the outputs to create a list of the maximum log likelihood model of each fit to our three images.

In [7]:
ml_vector = [samps.max_log_likelihood_vector for samps in agg.values("samples")]

print("Max Log Likelihood Model Parameter Lists: \n")
print(ml_vector, "\n\n")

Max Log Likelihood Model Parameter Lists: 

[[-0.0015885303373883138, 0.0003366972283607333, 0.05248045793702862, 0.000482727772216379, 1.5938014065752786, -1.258210617982979e-05, -0.001332392726817694, 0.0526836853815417, 0.00014220180508696598, 0.9999516107495982, 0.8005621575704052, 4.000203100627748], [-0.002810175615195956, -0.0011918626355703712, 0.0527376007895435, 0.004969297327886605, 1.5924463495158379, -0.0013527310654571673, 0.0003761767229535174, 0.05250889950479592, 4.804996614819311e-06, 0.999252145142791, 0.7993627156014809, 4.000052675016967], [0.000583811625038064, 0.00027701618192760925, 0.05278890212043312, -0.003479241033678737, 1.5953126885375088, 0.0005880087171205556, 0.0008615961360425253, 0.052534563594212755, 0.00023085325506054313, 1.0000881114528457, 0.799178400032618, 4.000086819777645]] 




This provides us with lists of all model parameters. However, this isn't that much use, which values correspond to 
which parameters?

The list of parameter names are available as a property of the `Model` included with the `Samples`, as are labels 
which can be used for labeling figures.

In [8]:
for samples in agg.values("samples"):
    model = samples.model
    print(model)
    print(model.parameter_names)
    print(model.parameter_labels)

Galaxy (centre_0, GaussianPrior, mean = 0.0, sigma = 0.001), (centre_1, GaussianPrior, mean = 0.0, sigma = 0.001), (elliptical_comps_0, UniformPrior, lower_limit = 0.052, upper_limit = 0.053), (elliptical_comps_1, UniformPrior, lower_limit = -0.005, upper_limit = 0.005), (einstein_radius, UniformPrior, lower_limit = 1.59, upper_limit = 1.61), Galaxy (centre_0, GaussianPrior, mean = 0.0, sigma = 0.001), (centre_1, GaussianPrior, mean = 0.0, sigma = 0.001), (elliptical_comps_0, GaussianPrior, mean = 0.0526, sigma = 0.0001), (elliptical_comps_1, GaussianPrior, mean = 0.0, sigma = 0.0001), (intensity, LogUniformPrior, lower_limit = 0.999, upper_limit = 1.0001), (effective_radius, LogUniformPrior, lower_limit = 0.799, upper_limit = 0.801), (sersic_index, UniformPrior, lower_limit = 3.999, upper_limit = 4.001)
['centre_0', 'centre_1', 'elliptical_comps_0', 'elliptical_comps_1', 'einstein_radius', 'centre_0', 'centre_1', 'elliptical_comps_0', 'elliptical_comps_1', 'intensity', 'effective_radi

These lists will be used later for visualization, how it is often more useful to create the model instance of every fit.

In [9]:
ml_instances = [samps.max_log_likelihood_instance for samps in agg.values("samples")]
print("Maximum Log Likelihood Model Instances: \n")
print(ml_instances, "\n")

Maximum Log Likelihood Model Instances: 

[<autofit.mapper.model.ModelInstance object at 0x7f8b5d1195b0>, <autofit.mapper.model.ModelInstance object at 0x7f8b7eeb6640>, <autofit.mapper.model.ModelInstance object at 0x7f8b5d119940>] 



A model instance contains all the model components of our fit, most importantly the list of galaxies we specified in 
the pipeline.

In [10]:
print(ml_instances[0].galaxies)
print(ml_instances[1].galaxies)
print(ml_instances[2].galaxies)

These galaxies will be named according to the search (in this case, `lens` and `source`).

In [11]:
print(ml_instances[0].galaxies.lens)
print()
print(ml_instances[1].galaxies.source)

Redshift: 0.5
Mass Profiles:
EllIsothermal
centre: (-0.0015885303373883138, 0.0003366972283607333)
elliptical_comps: (0.05248045793702862, 0.000482727772216379)
axis_ratio: 0.9002688042110568
angle: 44.73649730685004
einstein_radius: 1.5938014065752786
slope: 2.0
core_radius: 0.0
id: 1
_is_frozen: False
_frozen_cache: {}
_assertions: []
cls: <class 'autogalaxy.profiles.mass_profiles.total_mass_profiles.EllIsothermal'>

Redshift: 1.0
Light Profiles:
EllSersic
centre: (-0.0013527310654571673, 0.0003761767229535174)
elliptical_comps: (0.05250889950479592, 4.804996614819311e-06)
axis_ratio: 0.9002214618073412
angle: 44.99737848223414
intensity: 0.999252145142791
effective_radius: 0.7993627156014809
sersic_index: 4.000052675016967
id: 30
_is_frozen: False
_frozen_cache: {}
_assertions: []
cls: <class 'autogalaxy.profiles.light_profiles.EllSersic'>


Their `LightProfile`'s and `MassProfile`'s are also named according to the search.

In [12]:
print(ml_instances[0].galaxies.lens.mass)
print(ml_instances[1].galaxies.source.bulge)

EllIsothermal
centre: (-0.0015885303373883138, 0.0003366972283607333)
elliptical_comps: (0.05248045793702862, 0.000482727772216379)
axis_ratio: 0.9002688042110568
angle: 44.73649730685004
einstein_radius: 1.5938014065752786
slope: 2.0
core_radius: 0.0
id: 1
_is_frozen: False
_frozen_cache: {}
_assertions: []
cls: <class 'autogalaxy.profiles.mass_profiles.total_mass_profiles.EllIsothermal'>
EllSersic
centre: (-0.0013527310654571673, 0.0003761767229535174)
elliptical_comps: (0.05250889950479592, 4.804996614819311e-06)
axis_ratio: 0.9002214618073412
angle: 44.99737848223414
intensity: 0.999252145142791
effective_radius: 0.7993627156014809
sersic_index: 4.000052675016967
id: 30
_is_frozen: False
_frozen_cache: {}
_assertions: []
cls: <class 'autogalaxy.profiles.light_profiles.EllSersic'>


We can access the `median pdf` model, which is the model computed by marginalizing over the samples of every parameter 
in 1D and taking the median of this PDF.

In [13]:
mp_vector = [samps.median_pdf_vector for samps in agg.values("samples")]
mp_instances = [samps.median_pdf_instance for samps in agg.values("samples")]

print("Median PDF Model Parameter Lists: \n")
print(mp_vector, "\n")
print("Most probable Model Instances: \n")
print(mp_instances, "\n")
print(mp_instances[0].galaxies.lens.mass)
print()

Median PDF Model Parameter Lists: 

[[-0.0015885303373883138, 0.0003366972283607333, 0.05248045793702862, 0.000482727772216379, 1.5938014065752786, -1.258210617982979e-05, -0.001332392726817694, 0.0526836853815417, 0.00014220180508696598, 0.9999516107495982, 0.8005621575704052, 4.000203100627748], [-0.002810175615195956, -0.0011918626355703712, 0.0527376007895435, 0.004969297327886605, 1.5924463495158379, -0.0013527310654571673, 0.0003761767229535174, 0.05250889950479592, 4.804996614819311e-06, 0.999252145142791, 0.7993627156014809, 4.000052675016967], [0.000583811625038064, 0.00027701618192760925, 0.05278890212043312, -0.003479241033678737, 1.5953126885375088, 0.0005880087171205556, 0.0008615961360425253, 0.052534563594212755, 0.00023085325506054313, 1.0000881114528457, 0.799178400032618, 4.000086819777645]] 

Most probable Model Instances: 

[<autofit.mapper.model.ModelInstance object at 0x7f8b9c31f2b0>, <autofit.mapper.model.ModelInstance object at 0x7f8b6e5ff7c0>, <autofit.mapper.m

We can compute the model parameters at a given sigma value (e.g. at 3.0 sigma limits).

These parameter values do not account for covariance between the model. For example if two parameters are degenerate 
this will find their values from the degeneracy in the `same direction` (e.g. both will be positive). we'll cover
how to handle covariance in a later tutorial.

Here, I use "uv3" to signify this is an upper value at 3 sigma confidence,, and "lv3" for the lower value.

In [14]:
uv3_vectors = [
    samps.vector_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

uv3_instances = [
    samps.instance_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

lv3_vectors = [
    samps.vector_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]

lv3_instances = [
    samps.instance_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]

print("Errors Lists: \n")
print(uv3_vectors, "\n")
print(lv3_vectors, "\n")
print("Errors Instances: \n")
print(uv3_instances, "\n")
print(lv3_instances, "\n")

Errors Lists: 

[[0.0020726395875329882, 0.0016989454900017038, 0.05297152212947684, 0.004700840757134899, 1.6096743658901331, 0.0018709830353378885, 0.0016622996492079985, 0.05286634763437211, 0.0002092154191389194, 1.0000471311060464, 0.8009938361280727, 4.000942950125111], [0.0018216280125214392, 0.0026829133010355406, 0.05298360455732448, 0.004969297327886605, 1.609474427415872, 0.002862208672064772, 0.002574504061070538, 0.05278704573113879, 0.0002751386324371285, 1.0000836155660253, 0.8009327236264946, 4.000991657601819], [0.003590615255239051, 0.0020481277060464056, 0.05299938368076129, 0.004920965730638435, 1.6092380220836926, 0.0033339270492173, 0.0033662207571026697, 0.05278961681975276, 0.00023085325506054313, 1.0000999770714611, 0.8009974578299766, 4.000926464483869]] 

[[-0.003414848011693293, -0.00290037779350947, 0.05203793435668934, -0.004828897594325253, 1.5905910699588657, -0.0018014159990123663, -0.001546362319571057, 0.05233301424581308, -0.00014786907271864398, 0.9

We can compute the upper and lower errors on each parameter at a given sigma limit.

Here, "ue3" signifies the upper error at 3 sigma. 

In [15]:
ue3_vectors = [
    samps.error_vector_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

ue3_instances = [
    samps.error_instance_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

le3_vectors = [
    samps.error_vector_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]
le3_instances = [
    samps.error_instance_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]

print("Errors Lists: \n")
print(ue3_vectors, "\n")
print(le3_vectors, "\n")
print("Errors Instances: \n")
print(ue3_instances, "\n")
print(le3_instances, "\n")

Errors Lists: 

[[0.003661169924921302, 0.0013622482616409705, 0.0004910641924482165, 0.0042181129849185196, 0.01587295931485455, 0.0018835651415177184, 0.0029946923760256925, 0.0001826622528304095, 6.701361405195343e-05, 9.552035644821988e-05, 0.0004316785576675697, 0.0007398494973633873], [0.004631803627717396, 0.0038747759366059118, 0.00024600376778097927, 0.0, 0.017028077900034244, 0.004214939737521939, 0.0021983273381170204, 0.00027814622634287495, 0.0002703336358223092, 0.0008314704232342818, 0.001570008025013725, 0.0009389825848522548], [0.003006803630200987, 0.0017711115241187964, 0.00021048156032817122, 0.008400206764317172, 0.013925333546183749, 0.002745918332096744, 0.0025046246210601443, 0.0002550532255400084, 0.0, 1.1865618615436446e-05, 0.0018190577973585809, 0.0008396447062235524]] 

[[0.001826317674304979, 0.0032370750218702034, 0.0004425235803392838, 0.005311625366541632, 0.003210336616412901, 0.0017888338928325365, 0.00021396959275336294, 0.00035067113572861963, 0.000

The maximum log likelihood of each model fit and its Bayesian log evidence (estimated via the nested sampling 
algorithm) are also available.

Given each fit is to a different image, these are not very useful. However, in a later tutorial we'll look at using 
the aggregator for images that we fit with many different models and many different pipelines, in which case comparing 
the evidences allows us to perform Bayesian model comparison!

In [16]:
print("Maximum Log Likelihoods and Log Evidences: \n")
print([max(samps.log_likelihoods) for samps in agg.values("samples")])
print([samps.log_evidence for samps in agg.values("samples")])

Maximum Log Likelihoods and Log Evidences: 

[-105353226.94727705, -46248444.52272034, -24278073.90682222]
[-105353231.61185512, -46248449.1872984, -24278078.57140029]


We can also print the "model_results" of all searches, which is string that summarizes every fit`s lens model providing 
quick inspection of all results.

In [17]:
results = agg.model_results
print("Model Results Summary: \n")
print(results, "\n")

Model Results Summary: 



Bayesian Evidence                                                                         -105353231.61185512
Maximum Likelihood                                                                        -105353226.94727705

Maximum Log Likelihood Model:

galaxies
    lens
        mass
            centre
                centre_0                                                                  -0.002
                centre_1                                                                  0.000
            elliptical_comps
                elliptical_comps_0                                                        0.052
                elliptical_comps_1                                                        0.000
            einstein_radius                                                               1.594
    source
        bulge
            centre
                centre_0                                                                  -0.000
                cent

The Probability Density Functions (PDF's) of the results can be plotted using the library:

 corner.py: https://corner.readthedocs.io/en/latest/

(In built visualization for PDF's and non-linear searches is a future feature of PyAutoFit, but for now you`ll have to 
use the libraries yourself!).

(uncomment the code below to make a corner.py plot.)

In [18]:
# import corner
#
# for samples in agg.values("samples"):
#
#     corner.corner(
#         xs=samples.parameters,
#         weights=samples.weights,
#         labels=samples.model.parameter_labels,
#     )

Finished.